## 数据清洗

大家好，这一期我将为大家带来我的pandas学习心得第二期：数据清理。这一步非常重要，一般在获取数据源之后，我们紧接着就要开始这一步，以便为了之后的各种操作，简单来说，我们的目标就是让数据看起来赏心悦目，规规矩矩的，所以我们会对原始的dataframe做一些必要的美容，包括规范命名，去除异常值，重新选择合适的index啊，处理缺失值，统一列的命名等等。

这一期我会和大家分享一些比较好用常见的清洗方法。首先还是让我们来简单看一下本文结构


### 

In [ ]:
import pandas as pd
import numpy as np